In [1]:
import numpy as np
import pandas as pd
# from skimage import imread
from matplotlib.image import imread
import plotly.express as px
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle
from sklearn.metrics import confusion_matrix, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models,callbacks,metrics,backend
from tensorflow.keras.optimizers import RMSprop
import sys
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
# sys.path.append(os.path.abspath(r'C:\Users\raulg\Documents\THEBRIDGE_DS\0.-Repo_Git\ml_alzheimer_class\src'))
sys.path.append(os.path.relpath('../src'))
import utils as ut
import evaluation as ev
import math


In [2]:
# model5 = models.load_model("../models/image/model_5.keras")
x1_train,x1_test,x1_val,y1_train,y1_test,y1_val,scal = ut.img_images_load('../data/images/processed_train/x1_64.pkl','../data/images/processed_train/y1_64.pkl',
                                                                     '../data/images/processed_val/x2_64.pkl','../data/images/processed_val/y2_64.pkl',
                                                                     val_set = True,reshuffle=True,img_size=64)


x1_train shape (34326, 64, 64, 1)
x1_test shape (6058, 64, 64, 1)
y1_train shape (34326,)
y1_test shape (6058,)
y1_train distribution: 
 [[    0 10880]
 [    1  9520]
 [    2  8377]
 [    3  5549]]
y1_test distribution: 
 [[   0 1920]
 [   1 1680]
 [   2 1479]
 [   3  979]]
x1_val shape (6866, 64, 64, 1)
y1_val shape (6866,)
y1_val distribution: 
 [[   0 2176]
 [   1 1904]
 [   2 1676]
 [   3 1110]]


In [ ]:
### COMP TIME: 23 minutes
model5 = models.Sequential()
model5.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model5.add(layers.Dropout(0.5))  # Dropout so overfitting is reduced
model5.add(layers.MaxPooling2D((2, 2)))
model5.add(layers.Conv2D(64, (3, 3), activation='relu'))
model5.add(layers.MaxPooling2D((2, 2)))
model5.add(layers.Flatten())

# Capa densa con Dropout
model5.add(layers.Dense(64, activation='relu'))
model5.add(layers.Dropout(0.5))  # Dropout so overfitting is reduced

model5.add(layers.Dense(4, activation='softmax'))

# Compilar el modelo
model5.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='sparse_categorical_crossentropy', #sparse_categorical_crossentropy
              metrics=['accuracy'])

model5.fit(x1_train,y1_train,batch_size=128,epochs=70,validation_data=(x1_val,y1_val), callbacks=callbacks.EarlyStopping(patience=5)
          )

In [4]:
# model4c.save("../models/image/model_5.keras")
model5 =  models.load_model("../models/image/model_5.keras")

In [5]:
y1_pred = model5.predict(x1_test).argmax(axis=1)

190/190 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step


In [20]:
confusion_matrix(y1_test, y1_pred,normalize='true').round(3) #,normalize='pred'

array([[0.76 , 0.157, 0.083, 0.001],
       [0.024, 0.87 , 0.102, 0.004],
       [0.003, 0.074, 0.92 , 0.003],
       [0.   , 0.003, 0.006, 0.991]])

In [7]:
recall_score(y1_test, y1_pred,average='weighted')

np.float64(0.8667877187190491)